In [225]:
# Set the seed value for the notebook so the results are reproducible

from numpy.random import seed
seed(1)

In [226]:
# import necessary libraries

import warnings
warnings.simplefilter('ignore')

import numpy as np

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_classification

In [227]:
# load pandas DataFrame

pd.options.display.max_columns = None    #shows all columns in dataframe

ncaaDF = pd.read_csv("NCAA_data.csv")
ncaaDF.head()

,player_id,name,class,college,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,kenyon-martin-1,Kenyon Martin,Player,cincinnati,142,292,0.587,0.586,0.494,0.581,116,81,2042.0,1279,124,0.222000,2.0,869,0.592,14.3,139.0,0.346,2.781587,5.719882,25.053869,2.428991,0.039177,17.022527,2.722821
1,aj-guyton-1,A.J. Guyton,Player,indiana,403,52,0.541,0.455,0.249,0.790,128,73,3260.0,2100,129,0.414000,283.0,429,0.572,13.6,218.0,0.145,4.944785,0.638037,25.766871,1.582822,3.472393,5.263804,2.674847
2,jake-voskuhl-1,Jake Voskuhl,Player,uconn,124,193,0.542,0.542,0.555,0.656,138,83,2184.0,863,86,0.329163,0.0,880,0.573,23.0,160.0,0.237,2.271062,3.534799,15.805861,1.575092,0.000000,16.117216,2.930403
3,khalid-el-amin-1,Khalid El-Amin,Bust,uconn,479,10,0.486,0.416,0.257,0.822,108,70,2149.0,1650,186,0.355000,195.0,319,0.527,15.6,188.0,0.158,8.915775,0.186133,30.711959,3.462075,3.629595,5.937645,3.499302
4,mike-smith-1,Mike Smith,Bust,louisiana monroe,147,42,0.507,0.428,0.357,0.769,55,80,1938.0,1062,90,0.318000,130.0,449,0.551,20.0,241.0,0.165,3.034056,0.866873,21.919505,1.857585,2.683179,9.267286,4.974200


In [228]:
# Split the x variables from the y variable.
# Features with # are INCLUDED (not dropped) indepedent variables

X = ncaaDF.drop(["player_id",
                 "name",
                 "class", 
                 "college", 
                 "assists", 
                 "blocks", 
                 "effective_field_goal_percentage",
                 #"field_goal_percentage",
                 #"free_throw_attempt_rate",
                 "free_throw_percentage",
                 "games_played",
                 #"height", 
                 "minutes_played",
                 "points", 
                 "steals", 
                 #"three_point_percentage",
                 "three_pointers",
                 "total_rebounds",
                 #"true_shooting_percentage",
                 "turnover_percentage",
                 "turnovers",
                 #"win_shares_per_40_minutes",
                 "assists_per_40",
                 "blocks_per_40",
                 "points_per_40",
                 "steals_per_40",
                 "three_pointers_per_40",
                 "total_rebounds_per_40",
                 "turnovers_per_40"
                ], axis=1)
y = ncaaDF["class"]
print(f"Number of Rows and Independent(X) Variable: {X.shape}, \n Number of Rows (One Dependent(Y) variable): {y.shape}")

Number of Rows and Independent(X) Variable: (853, 6), 
 Number of Rows (One Dependent(Y) variable): (853,)


In [229]:
# Label encoding for Dependent(Y) Variable

label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original 

------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label:

Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Bust
Encoded Label: 0
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Player
Encoded Label: 1
------------
Original Class: Bust
Enc

In [230]:
# Scale the Independent(X) Variables

X_scaler = StandardScaler().fit(X_train)

#X_train_scaled = X_scaler.transform(X_train)
# X_test_scaled = X_scaler.transform(X_test)

X_scaled = X_scaler.transform(X)

encoded_y = encoded_y   #Use this Y array for the KFold cell far below

In [231]:
# Create a normal neural network with 12 inputs (12 independent variables), 6 hidden nodes, and 4 outputs
# input_dim refers to number of independent(X) variables
# units is output shape i.e. number of categories in the Dependent Variable Column
# In this example, final units=4 because Four Rankings: Star, Above Average, Below Average, Bust

model = Sequential()
model.add(Dense(units=30, activation='relu', input_dim=X.shape[1]))
model.add(Dense(units=25, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=len(y.unique()), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_59 (Dense)             (None, 30)                210       
_________________________________________________________________
dense_60 (Dense)             (None, 25)                775       
_________________________________________________________________
dense_61 (Dense)             (None, 20)                520       
_________________________________________________________________
dense_62 (Dense)             (None, 2)                 42        
Total params: 1,547
Trainable params: 1,547
Non-trainable params: 0
_________________________________________________________________


In [232]:
# Compile the model

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
             )

In [233]:
# # scikit-learn k-fold cross-validation (making multiple training datasets)

kfold_scores = []
kf = KFold(n_splits=5, random_state=1, shuffle=True)   # Define the split - into 10 folds 
kf.get_n_splits(ncaaDF)   # returns the number of splitting iterations in the cross-validator

for train_index, test_index in kf.split(X):
    #print("Train Index:", train_index) 
    #print("Test Index:", test_index)
    X_train, X_test, y_train, y_test = X_scaled[train_index], X_scaled[test_index], encoded_y[train_index], encoded_y[test_index]
    model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=10,
    shuffle=True,
    verbose=0
    )
    kfold_scores.append(model.evaluate(X_test, y_test))

170/170 [==============================] - 0s 41us/sample - loss: 0.5898 - acc: 0.7235


In [235]:
ynew = model.predict_classes(Xnew)
label_encoder.inverse_transform(ynew)

#counts_elements = np.unique(ynew, return_counts=True)

#rint(f"# of Busts(s): {counts_elements[0]} \n# of Players(s): {counts_elements[1]}")

array(['Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player', 'Player', 'Player', 'Player',
       'Player', 'Player', 'Player'], dtype=object)

In [236]:
Xnew

,field_goal_percentage,free_throw_attempt_rate,height,three_point_percentage,true_shooting_percentage,win_shares_per_40_minutes
0,0.417,0.332,77,0.329000,0.533,0.1680
1,0.769,1.038,85,0.000000,0.760,0.4440
2,0.648,0.306,81,0.400000,0.685,0.2450
3,0.392,0.349,81,0.313000,0.497,0.0890
4,0.422,0.308,75,0.350000,0.548,0.1850
5,0.568,0.425,82,0.308000,0.597,0.1820
6,0.351,0.415,75,0.338000,0.475,0.1170
7,0.586,0.606,82,0.417000,0.610,0.2990
8,0.509,0.175,77,0.424000,0.644,0.1680
9,0.606,0.507,81,0.000000,0.638,0.2650


In [216]:
# Save the model

model.save("Draft_Machine_NeuralNetwork.h5")

In [221]:
# Load the Draft_Data into Pandas

draftDF = pd.read_csv("Draft_data.csv")
draftDF.head()

,player_id,name,college,assists,blocks,effective_field_goal_percentage,field_goal_percentage,free_throw_attempt_rate,free_throw_percentage,games_played,height,minutes_played,points,steals,three_point_percentage,three_pointers,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,win_shares_per_40_minutes,assists_per_40,blocks_per_40,points_per_40,steals_per_40,three_pointers_per_40,total_rebounds_per_40,turnovers_per_40
0,anthony-edwards-2,Anthony Edwards,missouri state,64,12,0.494,0.417,0.332,0.737,22,77,694,438,30,0.329,55,115,0.533,12.2,57,0.168,3.688761,0.691643,25.244957,1.729107,3.170029,6.628242,3.285303
1,james-wiseman-1,James Wiseman,memphis,1,9,0.769,0.769,1.038,0.704,3,85,69,59,1,0.000,0,32,0.760,7.2,3,0.444,0.579710,5.217391,34.202899,0.579710,0.000000,18.550725,1.739130
2,obadiah-toppin-1,Obi Toppin,dayton,109,57,0.675,0.648,0.306,0.718,55,81,1556,909,43,0.400,32,361,0.685,14.8,115,0.245,2.802057,1.465296,23.367609,1.105398,0.822622,9.280206,2.956298
3,jaden-mcdaniels-1,Jaden McDaniels,washington,46,33,0.448,0.392,0.349,0.753,22,81,698,269,18,0.313,26,129,0.497,21.0,72,0.089,2.636103,1.891117,15.415473,1.031519,1.489971,7.392550,4.126074
4,nico-mannion-1,Nico Mannion,arizona,119,0,0.498,0.422,0.308,0.849,21,75,645,298,23,0.350,36,56,0.548,16.1,52,0.185,7.379845,0.000000,18.480620,1.426357,2.232558,3.472868,3.224806


In [223]:
# Set Xnew to have the same independent variables as X
Xnew = draftDF.drop(["player_id",
                 "name",
                 "college", 
                 "assists", 
                 "blocks", 
                 "effective_field_goal_percentage",
                 #"field_goal_percentage",
                 #"free_throw_attempt_rate",
                 "free_throw_percentage",
                 "games_played",
                 #"height", 
                 "minutes_played",
                 "points", 
                 "steals", 
                 #"three_point_percentage",
                 "three_pointers",
                 "total_rebounds",
                 #"true_shooting_percentage",
                 "turnover_percentage",
                 "turnovers",
                 #"win_shares_per_40_minutes",
                 "assists_per_40",
                 "blocks_per_40",
                 "points_per_40",
                 "steals_per_40",
                 "three_pointers_per_40",
                 "total_rebounds_per_40",
                 "turnovers_per_40"
                ], axis=1)

Xnew.head()

,field_goal_percentage,free_throw_attempt_rate,height,three_point_percentage,true_shooting_percentage,win_shares_per_40_minutes
0,0.417,0.332,77,0.329,0.533,0.168
1,0.769,1.038,85,0.000,0.760,0.444
2,0.648,0.306,81,0.400,0.685,0.245
3,0.392,0.349,81,0.313,0.497,0.089
4,0.422,0.308,75,0.350,0.548,0.185


In [224]:
# Predict the new Dependent Values

model = load_model("Draft_Machine_NeuralNetwork.h5")
ynew = model.predict_classes(Xnew)
label_encoder.inverse_transform(ynew)

counts_elements = np.unique(ynew, return_counts=True)

print(f"# of Busts(s): {counts_elements[0]} \n# of Players(s): {counts_elements[1]}")

# of Busts(s): [1] 
# of Players(s): [51]
